<a href="https://colab.research.google.com/github/Joseph669/TestRepo/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# 挂载Google Drive
!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("/content/drive/HRV")
!ls

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
from get_feats_csv import get_feats_file

get_feats_file()


In [0]:
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample

file_name = 'feats_data.csv'
#file_name = 'test_feats_data.csv'
# feats_tuple     = sample(range(0,10), 3)
feats_tuple     = (0,1,7,13,14)

file_data = np.array(pd.read_csv(file_name))
feats     = file_data[:, feats_tuple]
#feats     = preprocessing.StandardScaler().fit_transform(feats)
label     = file_data[:, -1]
x_train,x_test,y_train,y_test = train_test_split(feats, label, test_size=0.3, random_state=2)
clf = SVC(C=900, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参


print('----------------------------------------------------------')        
print('----------------------------------------------------------')
print('train accuracy:' + str(clf.score(x_train, y_train)))
print(confusion_matrix(y_train, clf.predict(x_train)))
print('test accuracy:' + str(clf.score(x_test, y_test)))
print(confusion_matrix(y_test, clf.predict(x_test)))


----------------------------------------------------------
----------------------------------------------------------
train accuracy:0.7142857142857143
[[20  2]
 [10 10]]
test accuracy:0.5555555555555556
[[7 1]
 [7 3]]


In [0]:
import numpy as np
import pandas as pd
import csv,os
from sklearn import preprocessing


output_file = 'subtraction.csv'
if os.path.exists(output_file):
    os.remove(output_file)
    
relax = (np.array(pd.read_csv('relax.csv')))[:, :]
stress = (np.array(pd.read_csv('stress.csv')))[:, :]

subtraction = stress - relax
#subtraction = preprocessing.MinMaxScaler().fit_transform(subtraction)
for row in subtraction:
    out = open(output_file, 'a', newline='')
    csv_write = csv.writer(out,dialect='excel')
    csv_write.writerow(row)



---

